In [1]:
# đặt biến môi trường 
%env SPARK_LOCAL_HOSTNAME=localhost

env: SPARK_LOCAL_HOSTNAME=localhost


In [2]:
#khởi động spark
import findspark
findspark.init()

In [3]:
# thêm các thư viện spark để thực hiện random 
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from pyspark.python.pyspark.shell import spark
import os

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.2.0
      /_/

Using Python version 3.9.7 (default, Sep 16 2021 16:59:28)
Spark context Web UI available at http://localhost:4041
Spark context available as 'sc' (master = local[*], app id = local-1642773769261).
SparkSession available as 'spark'.


In [4]:
df = spark.read.load("C:/Users/Admin/Documents/mushrooms.csv", format="csv", header=True, delimiter=",")
df

DataFrame[class: string, cap-shape: string, cap-surface: string, cap-color: string, bruises: string, odor: string, gill-attachment: string, gill-spacing: string, gill-size: string, gill-color: string, stalk-shape: string, stalk-root: string, stalk-surface-above-ring: string, stalk-surface-below-ring: string, stalk-color-above-ring: string, stalk-color-below-ring: string, veil-type: string, veil-color: string, ring-number: string, ring-type: string, spore-print-color: string, population: string, habitat: string]

In [5]:
df.show()

+-----+---------+-----------+---------+-------+----+---------------+------------+---------+----------+-----------+----------+------------------------+------------------------+----------------------+----------------------+---------+----------+-----------+---------+-----------------+----------+-------+
|class|cap-shape|cap-surface|cap-color|bruises|odor|gill-attachment|gill-spacing|gill-size|gill-color|stalk-shape|stalk-root|stalk-surface-above-ring|stalk-surface-below-ring|stalk-color-above-ring|stalk-color-below-ring|veil-type|veil-color|ring-number|ring-type|spore-print-color|population|habitat|
+-----+---------+-----------+---------+-------+----+---------------+------------+---------+----------+-----------+----------+------------------------+------------------------+----------------------+----------------------+---------+----------+-----------+---------+-----------------+----------+-------+
|    p|        x|          s|        n|      t|   p|              f|           c|        n|   

In [6]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df) for column in df.columns ]

stages = indexers
pipeline = Pipeline().setStages(stages)
df_r = pipeline.fit(df).transform(df)

clolumnsToDrop = ['class','cap-shape','cap-surface','cap-color','bruises','odor','gill-attachment','gill-spacing','gill-size','gill-color','stalk-shape','stalk-root','stalk-surface-above-ring','stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring','veil-type','veil-color','ring-number','ring-type','spore-print-color','population','habitat']
df_r = df_r.drop(*clolumnsToDrop)
df_r.show()

+-----------+---------------+-----------------+---------------+-------------+----------+---------------------+------------------+---------------+----------------+-----------------+----------------+------------------------------+------------------------------+----------------------------+----------------------------+---------------+----------------+-----------------+---------------+-----------------------+----------------+-------------+
|class_index|cap-shape_index|cap-surface_index|cap-color_index|bruises_index|odor_index|gill-attachment_index|gill-spacing_index|gill-size_index|gill-color_index|stalk-shape_index|stalk-root_index|stalk-surface-above-ring_index|stalk-surface-below-ring_index|stalk-color-above-ring_index|stalk-color-below-ring_index|veil-type_index|veil-color_index|ring-number_index|ring-type_index|spore-print-color_index|population_index|habitat_index|
+-----------+---------------+-----------------+---------------+-------------+----------+---------------------+----------

In [7]:
assem = VectorAssembler(inputCols=["class_index", "odor_index"], outputCol='features')
df_r = assem.transform(df_r)

In [8]:
labelIndexer = StringIndexer(inputCol="class_index", outputCol="indexedLabel").fit(df_r)

featureIndexer =\
                VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(df_r)

In [9]:
(trainingData, testData) = df_r.randomSplit([0.8, 0.2])

In [12]:
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

# Chuyển label
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

y_true = df_r.select("cap-color_index").rdd.flatMap(lambda x: x).collect()
y_pred = df_r.select("odor_index").rdd.flatMap(lambda x: x).collect()

pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Train model.
model = pipeline.fit(trainingData)

# dự đoán
predictions = model.transform(testData)


predictions.select("predictedLabel", "class_index", "features").show(5)

evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(predictions)

confusionmatrix = confusion_matrix(y_true, y_pred)

precision = precision_score(y_true, y_pred, average='micro')

recall = recall_score(y_true, y_pred, average='micro')

rfModel = model.stages[2]
print(rfModel)
print("Random Forest - Test Accuracy = %g" % (accuracy))
print("Random Forest - Test Error = %g" % (1.0 - accuracy))

print("The Confusion Matrix for Random Forest Model is :\n" + str(confusionmatrix))

print("The precision score for Random Forest Model is: " + str(precision))

print("The recall score for Random Forest Model is: " + str(recall))

+--------------+-----------+--------+
|predictedLabel|class_index|features|
+--------------+-----------+--------+
|           0.0|        0.0|   [0.0]|
|           0.0|        0.0|   [0.0]|
|           0.0|        0.0|   [0.0]|
|           0.0|        0.0|   [0.0]|
|           0.0|        0.0|   [0.0]|
+--------------+-----------+--------+
only showing top 5 rows

RandomForestClassificationModel: uid=RandomForestClassifier_bb6fc3ff7b22, numTrees=10, numClasses=2, numFeatures=1
Random Forest - Test Accuracy = 1
Random Forest - Test Error = 0
The Confusion Matrix for Random Forest Model is :
[[1184  288  288  288   48   48  128    0   12    0]
 [1032  744    0    0    0    0    0   64    0    0]
 [ 624  288  288  288    0    0    0    0   12    0]
 [  24  648    0    0  200  200    0    0    0    0]
 [ 448   96    0    0  152  152  128   64    0    0]
 [  72   96    0    0    0    0    0    0    0    0]
 [  80    0    0    0    0    0    0   64    0    0]
 [  32    0    0    0    0    0 